In [1]:
import numpy as np
import dicom
import sys
sys.path.append('..')
import cv2
import os
from AlgoEngine.sts import getSTSHistogram
from AlgoEngine.similarity import getSTSEmd
from AlgoEngine.utils import getVolume, getContours, getImageBlock, convertROIToCTSpace
from General.testing_utils import getContourInputs
from math import sqrt
import time
import matplotlib.pyplot as plt
start_time = time.time() # For runtime test


## Inputs to Function

Our inputs are two generated OVHs- One for the Query Patient (`UCLA_PR_5`) and one for the historical patient
(`'UCLA_PR_6'`). We currently compare OVH for a single PTV-OAR pair.

In [2]:
BASE_DIR = '/home/radiation/RadiationTherapyDecisionSupport/data/'
StudyID = 'UCLA_PR_5'
n_bins = 10
_, sop_ids = getImageBlock(StudyID, BASE_DIR)

ctFilenames = [fl for fl in os.listdir(BASE_DIR + StudyID) if 'CT.' in fl]
numImages = len(ctFilenames)

sampleCTImage = dicom.read_file(BASE_DIR + StudyID + '/' + ctFilenames[0])
width = sampleCTImage.Columns
height = sampleCTImage.Rows
row_spacing = float(sampleCTImage.PixelSpacing[0])
column_spacing = float(sampleCTImage.PixelSpacing[1])
slice_spacing = float(sampleCTImage.SliceThickness)

In [3]:
ROI_NAME = 'PTV'
block_shape, contour_data, image_orientation, image_position, pixel_spacing = getContourInputs(BASE_DIR, StudyID, ROI_NAME, excluding=[])
_, ptv_roi_block = getContours(block_shape, contour_data, image_orientation, image_position, pixel_spacing)
ptv_roi_block = convertROIToCTSpace(ptv_roi_block, image_position, sop_ids)

ROI_NAME = 'Bladder'
block_shape, contour_data, image_orientation, image_position, pixel_spacing = getContourInputs(BASE_DIR, StudyID, ROI_NAME, excluding=[])
_, oar_roi_block = getContours(block_shape, contour_data, image_orientation, image_position, pixel_spacing)
oar_roi_block = convertROIToCTSpace(oar_roi_block, image_position, sop_ids)

_, _, _, query_sts = getSTSHistogram(ptv_roi_block, oar_roi_block, n_bins)

In [4]:
StudyID = 'UCLA_PR_6'
n_bins = 10
_, sop_ids = getImageBlock(StudyID, BASE_DIR)

ctFilenames = [fl for fl in os.listdir(BASE_DIR + StudyID) if 'CT.' in fl]
numImages = len(ctFilenames)

sampleCTImage = dicom.read_file(BASE_DIR + StudyID + '/' + ctFilenames[0])
width = sampleCTImage.Columns
height = sampleCTImage.Rows
row_spacing = float(sampleCTImage.PixelSpacing[0])
column_spacing = float(sampleCTImage.PixelSpacing[1])
slice_spacing = float(sampleCTImage.SliceThickness)

ROI_NAME = 'PTV'
block_shape, contour_data, image_orientation, image_position, pixel_spacing = getContourInputs(BASE_DIR, StudyID, ROI_NAME, excluding=[])
_, ptv_roi_block = getContours(block_shape, contour_data, image_orientation, image_position, pixel_spacing)
ptv_roi_block = convertROIToCTSpace(ptv_roi_block, image_position, sop_ids)

ROI_NAME = 'Bladder'
block_shape, contour_data, image_orientation, image_position, pixel_spacing = getContourInputs(BASE_DIR, StudyID, ROI_NAME, excluding=[])
_, oar_roi_block = getContours(block_shape, contour_data, image_orientation, image_position, pixel_spacing)
oar_roi_block = convertROIToCTSpace(oar_roi_block, image_position, sop_ids)

_, _, _, historical_sts = getSTSHistogram(ptv_roi_block, oar_roi_block, n_bins)

## Function Here

In [5]:
emd  = getSTSEmd(query_sts, historical_sts)

## Testing

We rely exclusively on OpenCV for accurate EMD implementation- we have minimal way to confirm if this is the correct EMD or if the dissimilarity is correct. So we just print the EMD value and some rudimentary tests:

1) The EMD of two identical histograms should be 0
2) Runtime

In [6]:
print(emd)

15.580533981323242


In [7]:
print( getSTSEmd(query_sts, query_sts) == 0)
print( getSTSEmd(historical_sts, historical_sts) == 0)

True
True


In [8]:
print("Runtime (in minutes + fraction) of functions (single PTV and single OAR): " + str((time.time() - start_time) / 60))

Runtime (in minutes + fraction) of functions (single PTV and single OAR): 3.6390061378479004
